# Setup

Import libs

In [1]:
import numpy
import pandas
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.preprocessing import Imputer 
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import GridSearchCV

Constants

In [2]:
RANDOM_SEED = 42
TOL = 1e-5
MAX_ITER = 1e5
target = 'KWH'

Load preprocessed data

In [3]:
train_data = pandas.read_csv('data/train_data.csv')
train_data.head()

,REGIONC,DIVISION,REPORTABLE_DOMAIN,TYPEHUQ,NWEIGHT,HDD65,CDD65,HDD30YR,CDD30YR,Climate_Region_Pub,...,IECC_Climate_Pub_3B-4B,IECC_Climate_Pub_3C,IECC_Climate_Pub_4A,IECC_Climate_Pub_4C,IECC_Climate_Pub_5A,IECC_Climate_Pub_5B-5C,IECC_Climate_Pub_6A-6B,IECC_Climate_Pub_7A-7B-7AK-8AK,IECC_Climate_Pub_nan,KWH
0,0.356433,-0.130712,-0.095887,-1.391557,7.909538,0.144107,-0.406764,0.310213,-0.422437,1.036457,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,23103
1,1.316109,1.618052,1.490477,-0.552874,-0.105539,0.533708,-1.056278,0.313309,-1.247470,1.777458,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,12018
2,0.356433,-0.130712,0.270197,-0.552874,-0.381506,-1.257247,1.148720,-1.235898,1.057337,0.295456,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9612
3,1.316109,1.618052,1.490477,1.124492,0.727638,0.412901,-0.940829,0.252261,-1.126112,1.777458,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,7154
4,1.316109,0.918547,0.880337,-0.552874,-0.823157,0.569518,-0.614750,0.604836,-0.581963,-1.186546,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,6414


In [4]:
test_data = pandas.read_csv('data/test_data.csv')

In [5]:
layout = pandas.read_csv('data/public_layout.csv', index_col='Variable Name')
layout.head()

,Variable Label,Variable Order in File,Variable Type,Length
Variable Name,,,,
DOEID,Unique identifier for each respondent,1,Character,5
REGIONC,Census Region,2,Numeric,8
DIVISION,Census Division,3,Numeric,8
REPORTABLE_DOMAIN,Reportable states and groups of states,4,Numeric,8
TYPEHUQ,Type of housing unit,5,Numeric,8


# Actual modeling

We're leveraging Scikit Learn for some basic algorithms that should give us decent results, at least as a benchmark. It also offers evaluation by R2 correlation, which is adequate for a regression case like this, so let's use that to compare a few different models

In [6]:
features = train_data.columns.drop(target)
assert(len(train_data.columns) - len(features) == 1)

### Linear regression, no regularization
(not promising... but a quick first try)

In [7]:
# initialize and fit model
model_linear_full = LinearRegression()
model_linear_full.fit(train_data[features], numpy.ravel(train_data[target]))

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [8]:
#How does it generalize?
model_linear_full.score(test_data[features], numpy.ravel(test_data[target]))

-9.785760207702073e+19

Wow. Hard to see a model so bad. But it can be overfitting, with this data/feature ratio, so let's try removing some features.

In [9]:
#Pruning features
linear_feature_selector_standard = SelectFromModel(model_linear_full, prefit=True, threshold='mean')
select_train_data_linear_standard = linear_feature_selector_standard.transform(train_data[features])
select_test_data_linear_standard = linear_feature_selector_standard.transform(test_data[features])
select_train_data_linear_standard.shape

(9664, 36)

Only 36 features have coefficients above the mean. Let's try using those.

In [10]:
#Fitting with selected features
model_linear_select_standard = LinearRegression()
model_linear_select_standard.fit(select_train_data_linear_standard, numpy.ravel(train_data[target]))
model_linear_select_standard.score(select_test_data_linear_standard, numpy.ravel(test_data[target]))

-1.241432175334158e+23

Way worse. But just to make sure, perhaps feature selection should have been even stricter. Let's try with a lower threshold than the mean.

In [11]:
#Pruning features
feature_threshold = 2.1*1e14
linear_feature_selector_restrictive = SelectFromModel(model_linear_full, prefit=True, threshold=feature_threshold)
select_train_data_linear_restrictive = linear_feature_selector_restrictive.transform(train_data[features])
select_test_data_linear_restrictive = linear_feature_selector_restrictive.transform(test_data[features])
select_train_data_linear_restrictive.shape

(9664, 21)

In [12]:
#Fitting with selected features
model_linear_select_restrictive = LinearRegression()
model_linear_select_restrictive.fit(select_train_data_linear_restrictive, numpy.ravel(train_data[target]))
model_linear_select_restrictive.score(select_test_data_linear_restrictive, numpy.ravel(test_data[target]))

0.14553103242031762

Some predictive power, but far from useful. Let's try something else.

### Decision Tree regression

In [13]:
#Fitting with all features
model_tree_full = DecisionTreeRegressor(random_state=RANDOM_SEED, criterion='friedman_mse') #friedman_mse criterion proves more consistent than others
model_tree_full.fit(train_data[features], numpy.ravel(train_data[target]))

DecisionTreeRegressor(criterion='friedman_mse', max_depth=None,
                      max_features=None, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, presort=False,
                      random_state=42, splitter='best')

In [14]:
#How does it generalize?
model_tree_full.score(test_data[features], numpy.ravel(test_data[target]))

0.5825027849166835

As expected for this case, much better than Linear Regression, by nature of the algorithm being a better fit to this data.

Still a long way to go. Perhaps some feature selection can add value? Reduce overfitting?

In [15]:
#Pruning features
tree_feature_selector_standard = SelectFromModel(model_tree_full, prefit=True)
select_train_data_tree_standard = tree_feature_selector_standard.transform(train_data[features])
select_test_data_tree_standard = tree_feature_selector_standard.transform(test_data[features])
select_train_data_tree_standard.shape

(9664, 30)

A mere 30 features above mean. Let's check results.

In [16]:
#Fitting with selected features
model_tree_select_standard = DecisionTreeRegressor(random_state=RANDOM_SEED,criterion='friedman_mse')
model_tree_select_standard.fit(select_train_data_tree_standard, numpy.ravel(train_data[target]))
model_tree_select_standard.score(select_test_data_tree_standard, numpy.ravel(test_data[target]))

0.6982857640989614

A lot better, with a much lighter model.

If a Decision Tree performed well, perhaps a Random Forest will do even better?

### Random forest

In [17]:
# Quick hyperparameter search - how many forests in the tree to optimize the score / time balance?
tune_parameters = {'n_estimators':[1,10,100,1000]}
model_forest = RandomForestRegressor(random_state=RANDOM_SEED, verbose=1)
cross_validation_forest = GridSearchCV(model_forest, tune_parameters, scoring='r2', return_train_score=False)
cross_validation_forest.fit(train_data[features], numpy.ravel(train_data[target]))
cross_validation_forest.cv_results_

C:\Users\vbfal\.conda\envs\ProjectZ\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed

{'mean_fit_time': array([  1.31092413,   9.16959222,  49.46776152, 494.24941214]),
 'std_fit_time': array([0.02013023, 2.96559284, 0.30510673, 1.60275121]),
 'mean_score_time': array([0.01800354, 0.02200143, 0.0770069 , 0.72271657]),
 'std_score_time': array([2.24783192e-06, 8.04165779e-03, 1.63745010e-03, 6.59985933e-03]),
 'param_n_estimators': masked_array(data=[1, 10, 100, 1000],
              mask=[False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'n_estimators': 1},
  {'n_estimators': 10},
  {'n_estimators': 100},
  {'n_estimators': 1000}],
 'split0_test_score': array([0.51523358, 0.8113915 , 0.82939925, 0.83245291]),
 'split1_test_score': array([0.59538272, 0.81447893, 0.8409781 , 0.84630467]),
 'split2_test_score': array([0.65782056, 0.79272343, 0.80602318, 0.80831055]),
 'mean_test_score': array([0.58947127, 0.80619849, 0.82546725, 0.82902307]),
 'std_test_score': array([0.05836226, 0.00961058, 0.01453797, 0.01569876]),
 'rank_test_s

In [18]:
#Fitting with all features - for the sake of balance between time and efficiency, let's go with 100 estimators
model_forest_full = RandomForestRegressor(random_state=RANDOM_SEED, criterion='friedman_mse',
                                          n_estimators=100)
model_forest_full.fit(train_data[features], numpy.ravel(train_data[target]))

RandomForestRegressor(bootstrap=True, criterion='friedman_mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=100,
                      n_jobs=None, oob_score=False, random_state=42, verbose=0,
                      warm_start=False)

In [19]:
#How does it generalize?
model_forest_full.score(test_data[features], numpy.ravel(test_data[target]))

0.8634460453481361

Considerably better than a single Decision Tree, as expected. Already something that can be used.

Can we reduce the number of features and keep the score high?

In [20]:
#Pruning features
forest_feature_selector_standard = SelectFromModel(model_forest_full, prefit=True)
select_train_data_forest_standard = forest_feature_selector_standard.transform(train_data[features])
select_test_data_forest_standard = forest_feature_selector_standard.transform(test_data[features])
select_train_data_forest_standard.shape

(9664, 36)

Similar order of magnitude as the Decision Tree. 

In [21]:
#Fitting with selected features
model_forest_select_standard = RandomForestRegressor(random_state=RANDOM_SEED, criterion='friedman_mse',
                                                     n_estimators=100)
model_forest_select_standard.fit(select_train_data_forest_standard, numpy.ravel(train_data[target]))
model_forest_select_standard.score(select_test_data_forest_standard, numpy.ravel(test_data[target]))

0.8683030569334925

Not much gain on performance, but model is 90% smaller.

### Ridge Regression 

In [22]:
# Another quick hyperparameter search
tune_parameters = {'alpha':[0.01, 0.1, 1, 10, 100]}
model_ridge = Ridge(random_state=RANDOM_SEED, tol=TOL, max_iter=MAX_ITER)
cross_validation_ridge = GridSearchCV(model_ridge, tune_parameters, scoring='r2', return_train_score=False)
cross_validation_ridge.fit(train_data[features], numpy.ravel(train_data[target]))
cross_validation_ridge.cv_results_

C:\Users\vbfal\.conda\envs\ProjectZ\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


{'mean_fit_time': array([0.089993  , 0.09033354, 0.08266679, 0.08899919, 0.08334422]),
 'std_fit_time': array([0.00963704, 0.00579266, 0.01155768, 0.00282744, 0.0066593 ]),
 'mean_score_time': array([0.00433771, 0.0043354 , 0.00400146, 0.00466935, 0.00400345]),
 'std_score_time': array([4.74799983e-04, 4.70527668e-04, 1.32507737e-06, 4.71146214e-04,
        2.43399824e-06]),
 'param_alpha': masked_array(data=[0.01, 0.1, 1, 10, 100],
              mask=[False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'alpha': 0.01},
  {'alpha': 0.1},
  {'alpha': 1},
  {'alpha': 10},
  {'alpha': 100}],
 'split0_test_score': array([0.83705703, 0.83721972, 0.83777963, 0.83970515, 0.83858617]),
 'split1_test_score': array([0.84470482, 0.84486461, 0.84581922, 0.84867093, 0.84715643]),
 'split2_test_score': array([0.87952809, 0.87953759, 0.87972853, 0.87977875, 0.87478885]),
 'mean_test_score': array([0.85376159, 0.85387225, 0.85444074, 0.85604992, 0.853508

Relatively close across alpha values. Let's take the theoretical best one.

In [23]:
#Fitting with all features
model_ridge_full = Ridge(random_state=RANDOM_SEED, tol=TOL, max_iter=MAX_ITER, alpha=10)
model_ridge_full.fit(train_data[features],numpy.ravel(train_data[target]))

Ridge(alpha=10, copy_X=True, fit_intercept=True, max_iter=100000.0,
      normalize=False, random_state=42, solver='auto', tol=1e-05)

In [24]:
#How does it generalize?
model_ridge_full.score(test_data[features], numpy.ravel(test_data[target]))

0.8668089944859985

On par with a Random Forest. Does a lighter model work as well?

In [25]:
#Pruning features
ridge_feature_selector_standard = SelectFromModel(model_ridge_full, prefit=True)
select_train_data_ridge_standard = ridge_feature_selector_standard.transform(train_data[features])
select_test_data_ridge_standard = ridge_feature_selector_standard.transform(test_data[features])
select_train_data_ridge_standard.shape

(9664, 141)

In [26]:
#Fitting with selected features
model_ridge_select_standard = Ridge(random_state=RANDOM_SEED, tol=TOL, max_iter=MAX_ITER, alpha=10)
model_ridge_select_standard.fit(select_train_data_ridge_standard, numpy.ravel(train_data[target]))
model_ridge_select_standard.score(select_test_data_ridge_standard, numpy.ravel(test_data[target]))

0.8663116730585897

Similar performance, but didn't get as much size benefit as the random forest model. Perhaps we can prune farther and still keep performance? Let's try to match the last random forest order of magnitude.

In [34]:
#Pruning features
feature_threshold = 7*1e2
ridge_feature_selector_restrictive = SelectFromModel(model_ridge_full, prefit=True, threshold=feature_threshold)
select_train_data_ridge_restrictive = ridge_feature_selector_restrictive.transform(train_data[features])
select_test_data_ridge_restrictive = ridge_feature_selector_restrictive.transform(test_data[features])
select_train_data_ridge_restrictive.shape

(9664, 34)

In [35]:
#Fitting with selected features
model_ridge_select_restrictive = Ridge(random_state=RANDOM_SEED, tol=TOL, max_iter=MAX_ITER, alpha=10)
model_ridge_select_restrictive.fit(select_train_data_ridge_restrictive, numpy.ravel(train_data[target]))
model_ridge_select_restrictive.score(select_test_data_ridge_restrictive, numpy.ravel(test_data[target]))

0.848165008881888

So the Ridge regression loses more performance than the Random Forest on similar pruning.

# Choosing model

Chosing between the Random Forest and the Ridge Regression at this point depends on the constraints present. Ridge Regression, even without pruning, runs inference one order of magnitude faster than the Random Forest, as indicated by the numbers above. However to match the performance it will require more data and computing power on inference time, so this might be a limitation.

# Next steps

1. Determine a theoretical upper performance limit for the model - there is a limit to the predictive power of modeling given a certain dataset. If current models are close to that limit, it may not be worth pursuing more complex modeling.


2. Given room to the theoretical performance, or its uncertainty, attempt two more laboursome / compute-heavy approaches:

    a. Detailed manual feature engineering, looking through the long list of variables and thinking of new, composed values to capture non-linear relationships
    
    b. More advanced non-linear modeling leveraging neural networks. With the current dataset type a regular feed-forward / MLP architecture could be tested, albeit nothing so far particularly feeds the belief that its performance would be better than the models above. If in the future we acquire sequence-like data (such as time-based data more granular than year for example), recurrent network variations might prove interesting.